In [17]:
###############---------------------Pipeline de Preprocesamiento-------------------####################
#1.Ingesta de Datos 
#2.Selección de Variables
#3️.Transformaciones y Tratamiento de Datos
#4️.Enriquecimiento de Información
#5️.Exportación de Datos Listos para Modelado

import pandas as pd
import holidays
import numpy as np
from sklearn.preprocessing import StandardScaler

class ContugasPipeline:
    """
    Pipeline de procesamiento de datos para análisis de anomalías en redes de gas.
    Se incluyen etapas de ingesta, transformación, suavizado de variabilidad, manejo de valores cero 
    y enriquecimiento con información relevante.
    """
    def __init__(self, file_path):
        """
        Inicializa la clase con la ruta del archivo de datos.

        :param file_path: Ruta del archivo de Excel con los datos.
        """
        self.file_path = file_path
        self.data = None

    def load_data(self):
        """
        Carga los datos desde múltiples hojas de un archivo de Excel y los concatena en un DataFrame único.

        - Cada hoja representa datos de un cliente.
        - Se añade una columna 'Cliente' para indicar la fuente de cada registro.
        """
        excel_data = pd.ExcelFile(self.file_path)
        dataframes = []

        for sheet_name in excel_data.sheet_names:
            df = pd.read_excel(self.file_path, sheet_name=sheet_name)
            df['Cliente'] = sheet_name
            dataframes.append(df)

        self.data = pd.concat(dataframes, ignore_index=True)
    
    def preprocess_data(self):
        """
        Selecciona variables clave y preprocesa datos.

        - Filtrado de columnas clave ('Fecha', 'Presion', 'Temperatura', 'Volumen', 'Cliente').
        - Conversión de Fecha a formato datetime.
        - Extracción de Mes y Año para análisis temporal.
        - Eliminación de valores faltantes.
        """
        selected_columns = ['Fecha', 'Presion', 'Temperatura', 'Volumen', 'Cliente']
        self.data = self.data[selected_columns]
        
        self.data['Fecha'] = pd.to_datetime(self.data['Fecha'], errors='coerce')
        self.data.dropna(inplace=True)
        
        # Extraer mes y año
        self.data['Mes'] = self.data['Fecha'].dt.month
        self.data['Año'] = self.data['Fecha'].dt.year

    def smooth_variability(self, window=7):
        """
        Suaviza la variabilidad en la presión usando una media móvil.

        - Se probaron ventanas de 7, 15 y 30 días, el análisis del notebook de preprocesamiento indicó que 7 días es óptimo.
        - Reduce el ruido sin perder tendencias críticas en los datos.
        
        :param window: Tamaño de la ventana de suavizado (por defecto, 7 días).
        """
        self.data['Presion_Suavizada'] = self.data['Presion'].rolling(window=window, center=True).mean()
        
        # Elimina columnas innecesarias si existen
        self.data.drop(['Presion_Suavizada_15', 'Presion_Suavizada_30'], axis=1, inplace=True, errors='ignore')

    def add_holidays(self):
        """
        Incorporación de días feriados en Perú.

        - Identifica días festivos en los años presentes en los datos.
        - Agrega una columna `Es_Feriado` para evaluar el impacto de días festivos.
        """
        years = self.data['Año'].unique()
        peru_holidays = holidays.Peru(years=years)
        holiday_dates = {date.strftime("%Y-%m-%d") for date in peru_holidays.keys()}
        self.data['Es_Feriado'] = self.data['Fecha'].dt.strftime("%Y-%m-%d").isin(holiday_dates)

    def categorize_volume_by_client(self):
        """
        Categorización del volumen de consumo por cliente mediante percentiles.
        """
        clientes = self.data['Cliente'].unique()
        
        for cliente in clientes:
            df_cliente = self.data[self.data['Cliente'] == cliente]
            p10 = df_cliente['Volumen'].quantile(0.10)
            p90 = df_cliente['Volumen'].quantile(0.90)

            self.data.loc[self.data['Cliente'] == cliente, 'Anomalia'] = df_cliente['Volumen'].apply(
                lambda x: 'Anómalo' if x < p10 or x > p90 else 'Normal'
            )

        # Crear columna binaria (1 para anómalo, 0 para normal)
        self.data['Anomalia_bin'] = self.data['Anomalia'].map({'Anómalo': 1, 'Normal': 0})

    def filter_final_columns(self):
        """
        Filtra las columnas finales requeridas en la salida.
        """
        required_columns = ['Presion', 'Temperatura', 'Volumen', 'Cliente', 'Mes', 'Año', 'Es_Feriado', 'Anomalia', 'Anomalia_bin']
        self.data = self.data[required_columns]

    def save_clean_data(self, output_path="salida/data_cleaned.csv"):
        """
        Guarda el dataset procesado en un archivo CSV.

        :param output_path: Ruta del archivo de salida (por defecto, 'data_cleaned.csv').
        """
        self.data.to_csv(output_path, index=False)

In [18]:
#Ejecutando el Pipeline
pipeline = ContugasPipeline(r"insumo\Contugas_Datos.xlsx")
pipeline.load_data()
pipeline.preprocess_data()
pipeline.smooth_variability()
pipeline.add_holidays()
pipeline.categorize_volume_by_client()
pipeline.filter_final_columns()
pipeline.save_clean_data()